# Algonauts + Net2Brain CCN23 Hackathon

## Roadmap

1. **Loading Models**: Load _Scene Classification_ and _Scene Parsing_ Artificial Neural Networks (ANNs) using `Net2Brain`.
2. **Extracting features from ANNs internal representations**: Extract model features from  these ANNs processing a subset of the `Algonauts Challenge` Dataset.
3. **Training encoding Models**: Build encoding models that predict brain data from the DNN features using `Net2Brain`.
4. **Plotting**: Visualize the results on ROIs.
5. **Other Models**: Try other DNNs available through `Net2Brain`.

## If working on Google Colab

### Install Net2Brain and relevant dependencies

In [ ]:
!pip install -U git+https://github.com/cvai-roig-lab/Net2Brain

In [ ]:
%load_ext autoreload
%autoreload 2

### Restart runtime and install _nilearn_

In [ ]:
!pip install nilearn==0.9.2

### Mount the workshop data on your drive runtime
Before running the tutorial code you need to select [this](https://t.ly/jkIu-) folder and go to `organize` and `add shortcut`. You will then need to create a shortcut (without copying or taking space) of the folder to a desired path in your Google Drive, from which you can read the content after mounting using `drive.mount()`.

Please don't forget to edit the `data_dir` variable below with the path on your Drive to this shortcut folder.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
data_dir = '/content/drive/MyDrive/put_data_here' #@param {type:"string"}

## If working locally

If you are running this tutorial locally you need to make sure you have downloaded the challenge data located in folder `\subj01`. You can download it from [here]((https://t.ly/jkIu-).

Then uncomment and edit the `data_dir` variable below with the path to the parent folder containing the `\subj01` data folder.

In [ ]:
data_dir = '.'

## General imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
from pathlib import Path
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as plc
import pandas as pd
from tqdm import tqdm
import torch

from nilearn import datasets, plotting

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch_device = torch.device(device)

## About Net2Brain

__Net2Brain__ allows you to use one of over 1000 Deep Neural Networks (DNNs) for your experiments comparing human brain activity with the activations of artificial neural networks. The DNNs in __Net2Brain__ are obtained from what we call different _netsets_, which are libraries that provide different pretrained models.

__Net2Brain__ provides access to the following _netsets_:
- [Standard torchvision](https://pytorch.org/vision/stable/models.html) (`standard`).
This netset is a collection of the torchvision models including models for image classification, pixelwise semantic segmentation, object detection, instance segmentation, person keypoint detection, video classification, and optical flow.
- [Timm](https://github.com/rwightman/pytorch-image-models#models) (`timm`).
A deep-learning library created by Ross Wightman that contains a collection of state-of-the-art computer vision models.
- [PyTorch Hub](https://pytorch.org/docs/stable/hub.html) (`pytorch`).
These models are accessible through the torch.hub API and are trained for different visual tasks. They are not included in the torchvision module.
- [Unet](https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/) (`unet`).
Unet also is available through the torch.hub.API and is trained for abnormality segmentation in brain MRI.
- [Taskonomy](https://github.com/StanfordVL/taskonomy) (`taskonomy`). A set of networks trained for different visual tasks, like Keypoint-Detection, Depth-Estimation, Reshading, etc. The initial idea for these networks was to find relationships between different visual tasks.
- [Slowfast](https://github.com/facebookresearch/pytorchvideo) (`pyvideo`).
These models are state-of-the-art video classification models trained on the Kinetics 400 dataset, acessible through the torch.hub API.
- [CLIP](https://github.com/openai/CLIP) (`clip`).
CLIP (Contrastive Language-Image Pre-Training) is a vision+language multimodal neural network trained on a variety of (image, text) pairs.
- [CorNet](https://github.com/dicarlolab/CORnet) (`cornet`).
A set of neural networks whose structure is supposed to resemble the one of the ventral visual pathway and therefore implements more recurrent connections that are commonplace in the VVS.
- [Detectron2](https://github.com/facebookresearch/Detectron) (`detectron2`).
Facebook AI Research's software system that implements state-of-the-art object detection algorithms, including Mask R-CNN. It covers models trained for object classification and detection such as instance, panoptic and keypoint detection.
- [VISSL](https://github.com/facebookresearch/vissl) (`vissl`).
VISSL provides reference implementation of a large number of self-supervision approaches.

---
---

**Net2Brain** has three main components:
1. **ANN Selection**
> Select models by specific architectures, training objectives, or pre-training data
2. **Feature Extraction**
> Expects images or videos in .jpg, .png, or .mp4 format
3. **Brain-ANN comparison**
> Using forward encoding or RSA [Subjects x ROIs x Stimuli Condition x Stimuli Condition]


## Unraveling brain functionality with ANNs

Research correlating brain and artificial neural network representations can be used to unravel brain functionality across both space and time.

In this hackathon we will explore how different brain areas (ROIs) correlate distinctively with ANNs trained on different visual tasks.


This exploration is inspired by the study of [Dwivedi et al. (2021)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1009267), where they used the __Taskonomy__ netset, that contains ANNs trained in the following visual tasks:

In [ ]:
display(Image.open(Path(data_dir) / 'subj01/misc' / 'task_similarity_tree.png'))

This study grouped the Taskonomy ANNs into three groups:

- __2D Tasks__: segment_unsup2d, inpainting, keypoints2d, jigsaw, autoencoding, denoising.

- __3D Tasks__: reshading , curvature, depth_euclidean, keypoints3d, normal.

- __Semantic Tasks__: class_object, class_scene, segment_semantic.

And they found the representations of each of these groups correlated more strongly with different visual ROIs:

In [ ]:
display(Image.open(Path(data_dir) / 'subj01/misc' / 'unravelingBrainFunc.png'))

## Step 0: Load the _Algonauts Challenge_ dataset

For this hackathon, we will use a sub-selection of the [Algonauts 2023](http://algonauts.csail.mit.edu/challenge.html) challenge dataset.

In [ ]:
subj = '01' # We will only use subject 01 for this hackathon

sml_stim = Path(data_dir) / f'subj{subj}' / 'sml_images'
sml_fmri = Path(data_dir) / f'subj{subj}' / 'sml_fmri'

In [ ]:
# Create sorted lists of image file names
sml_img_list = os.listdir(sml_stim)
sml_img_list.sort()
print(f'Total images: {len(sml_img_list)}')

## Step 1: Selecting a pretrained ANN with Net2Brain





Pick one of the following three models by hackathon group:

In [ ]:
# model_name = 'segment_unsup2d' # For 2D features
model_name = 'reshading'       # For 3D features
# model_name = 'class_object'    # For Semantic features

To extract activations from a pretrained model of one of the netsets available with Net2Brain, you must first initialize the `FeatureExtractor` class and specify the name of the model as well as the netset it belongs to.

__Note__: If you want to implement your _own_ pre-trained ANN, use the model as an argument of `Feature Extractor` like:
`FeatureExtractor(model=my_model, device='cuda')`

In [ ]:
from net2brain.feature_extraction import FeatureExtractor

fx_model = FeatureExtractor(model=model_name,
                            netset='Taskonomy',
                            device=device)

Note that by default Net2Brain selects which layers of the model are going to be used to extract the features from.
To view the layers that are set to be extracted, you can inspect the attribute `layers_to_extract`, like:

In [ ]:
fx_model.layers_to_extract

You can also specify which layers you want to extract from the model.

To view a complete list of all available layers, you can use the class method `get_all_layers()` and overwrite the `layers_to_extract` argument with your desired subset.

In [ ]:
fx_model.get_all_layers()

## Step 2: Extracting interal representations from the ANN
To extract the activations from the specified layers, you can use the `extract()` method and provide the path to the images that you want to run through the network.
You can choose to save the features in different formats (numpy arrays, pytorch tensors, or into the `dataset` class from the [rsa](https://rsatoolbox.readthedocs.io/en/stable/) toolbox).

For this tutorial, we will use the numpy array (`npz`) format.


In [ ]:
# Create features for the chosen model
ft_path = f'sml_feats_{model_name}'
fx_model.extract(data_path=sml_stim,
                 save_path=ft_path, consolidate_per_layer=False,
                 layers_to_extract=['layer4'])

## Step 3: Building Encoding models


To find how well the ANN's features predict brain activity, we will train an encoding model using a 3-fold cross-validation process.

For every fold, we will:
1. Split the data into training/validation sets.
2. Use PCA to reduce the feature dimentions.
3. Train a regression model predicting voxel activations from ANN's representations.

We can train this model using the function the class `encoding` from Net2Brain.
The training will take some minutes.

In [ ]:
import net2brain.evaluations.encoding as encoding

roi_path = str(sml_fmri)
model_brain_df, model_brain_corr = encoding.linear_encoding(ft_path, roi_path,
                                                            model_name,
                                                            trn_tst_split=0.8,
                                                            n_folds=3,
                                                            n_components=70,
                                                            batch_size=300,
                                                            return_correlations=True)

In [ ]:
model_brain_df[['ROI', 'R']]

In [ ]:
## Save correlation values
# np.save(f'model_name_corr.npy', model_brain_corr)

## Save dataframe
model_brain_df.to_json(f'{model_name}_df.json')

## Step 4: Plot correlations

We will plot the ANN-Brain correlation results using Net2Brain and _nilearn_.

In [ ]:
d3_df = pd.read_json(Path(data_dir) / f'subj{subj}' / 'reshading_df.json')
sm_df = pd.read_json(Path(data_dir) / f'subj{subj}' / 'class_object_df.json')
d2_df = pd.read_json(Path(data_dir) / f'subj{subj}' / 'segment_unsup2d_df.json')

In [ ]:
model_brain_df

In [ ]:
from net2brain.evaluations.plotting import Plotting

# Plotting with significance
plotter = Plotting([d2_df, d3_df, sm_df])
# plotter = Plotting([model_brain_df])

In [ ]:
results_df = plotter.plot(metric="R")

In [ ]:
#results_df = plotter.plot(metric="R")

### Visualize the correlations using _nilearn_

We will load the ROI indices extracted as seen in the developer-kit

In [ ]:
roi_idx = np.load((Path(data_dir) / f'subj{subj}' / 'sml_roi_idx_map.npy'), allow_pickle=True)[()]

And we will load the subject to fsaverage projection, resulting in 163842 brain vertices.

In [ ]:
fsaverage = datasets.fetch_surf_fsaverage('fsaverage')

masks_dir = Path(data_dir) / f'subj{subj}' / 'roi_masks'
rh_fsaverage = np.load((masks_dir / 'rh.all-vertices_fsaverage_space.npy'), allow_pickle=True)
print(f'Shape of fsaverage: {rh_fsaverage.shape}')

In [ ]:
fs_idx = np.where(rh_fsaverage)[0]

In [ ]:
model_brain_dict = dict(zip(model_brain_df.ROI, model_brain_df.R))

In [ ]:
plot_data = np.zeros(rh_fsaverage.shape)
plot_data[fs_idx[roi_idx['V1']]]  = np.ones(fs_idx[roi_idx['V1']].shape)* model_brain_dict['rh_V1_fmri']
plot_data[fs_idx[roi_idx['V2']]]  = np.ones(fs_idx[roi_idx['V2']].shape)* model_brain_dict['rh_V2_fmri']
plot_data[fs_idx[roi_idx['V3']]]  = np.ones(fs_idx[roi_idx['V3']].shape)* model_brain_dict['rh_V3_fmri']
plot_data[fs_idx[roi_idx['PPA']]] = np.ones(fs_idx[roi_idx['PPA']].shape) * model_brain_dict['rh_PPA_fmri']

You can use `flat_right` and `curv_right` to vizualize a flatmap instead of cortical view:

In [ ]:
view = plotting.view_surf(
    surf_mesh=fsaverage['infl_right'],
    surf_map=plot_data, bg_map=fsaverage['sulc_right'],
    threshold=1e-14, colorbar=True, symmetric_cmap= False,
    cmap=plt.get_cmap('twilight_shifted')
)
view

### Which taskonomy group was the most correlated with different ROIs?
<b style='color:#0000ff;'>2D Tasks </b>, <b style='color:#00ff00;'>3D Tasks </b>, or   <b style='color:#ff00ff;'>Semantic Tasks </b>?

In [ ]:
taskonomy_cmap = plc.LinearSegmentedColormap.from_list("", ["#0000ff","#00ff00","#ff00ff"])
taskonomy_corr_dict =  {roi : [d2_df.R[ii],d3_df.R[ii],sm_df.R[ii]] for ii,roi in enumerate(d2_df.ROI) }
plot_data = np.zeros(rh_fsaverage.shape)
for roi in roi_idx.keys():
    plot_data[fs_idx[roi_idx[roi]]]  = np.ones(fs_idx[roi_idx[roi]].shape) * \
    (np.argmax(taskonomy_corr_dict['rh_'+roi+'_fmri'])+1e-10)
view = plotting.view_surf(fsaverage.infl_right, plot_data, bg_map=fsaverage.sulc_right, threshold=1e-14,
                          cmap=taskonomy_cmap, colorbar=False, symmetric_cmap=False)
view